In [1]:
# Import Packages
import pandas as pd, numpy as np
import os, sys, glob, re
from pathlib import Path
from itertools import compress
import math

In [2]:
exactmatch_yes_filepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.3_exact_matches\20210101-20220617\exactmatch_yes.csv")
fuzzymatch_filledin_filepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.4_fuzzy_matches\20210101-20220617\fuzzymatch_manually_filled_in.xlsx")
compustathassan_filepath = Path(r"C:\Users\jasonjia\Dropbox\Projects\conference_call\output\04_match_firm_names_to_gvkeys\04.1_process_compustat_and_hassan_files\compustat_and_hassan_firm_names_withgvkeyandcountry\20220705\compustat_and_hassan_firm_names_withgvkeyandcountry.csv")

In [3]:
# Import files
exactmatch = pd.read_csv(exactmatch_yes_filepath)
fuzzymatch = pd.read_excel(fuzzymatch_filledin_filepath)
fuzzymatch = fuzzymatch.rename(columns = {'clean_firm_name': "clean_firm_name_entryfile"})
compustathassan = pd.read_csv(compustathassan_filepath)
compustathassan = compustathassan.rename(columns = {'clean_firm_name': "clean_firm_name_compustathassan"})

In [4]:
exactmatch.head(1)

,clean_firm_name,count,gvkey,company_name,hqcountry
0,CAMDEN PROPERTY TRUST,41,28629.0,Camden Property Trust,US


In [5]:
fuzzymatch.head(1)

,clean_firm_name_entryfile,count,choice_1_name,choice_1_score,choice_2_name,choice_2_score,choice_3_name,choice_3_score,best_choice
0,EUROPEAN RELIANCE GENERAL INSURANCE CO S A,1,EUROPEAN RELIANCE GENERAL INSURANCE CO SA,98.795181,A E PLASTIK PAK INC,85.5,A M FOOD SERVICES INC,85.5,1.0


In [6]:
compustathassan.head(1)

,gvkey,company_name,hqcountry,clean_firm_name_compustathassan
0,1000.0,A & E Plastik Pak Inc.,US,A E PLASTIK PAK INC


In [7]:
# View value counts of manual matching
one_count = fuzzymatch[fuzzymatch['best_choice'] == 1].shape[0]
two_count = fuzzymatch[fuzzymatch['best_choice'] == 2].shape[0]
three_count = fuzzymatch[fuzzymatch['best_choice'] == 3].shape[0]
nan_count = fuzzymatch['best_choice'].isna().sum()

print("Number of entries in fuzzymatch:", fuzzymatch.shape[0])
print("Number of 1s in fuzzymatch:", one_count)
print("Number of 2s in fuzzymatch:", two_count)
print("Number of 3s in fuzzymatch:", three_count)
print("Number of non-matches in fuzzymatch:", nan_count)
assert(one_count + two_count + three_count + nan_count == fuzzymatch.shape[0])

Number of entries in fuzzymatch: 297
Number of 1s in fuzzymatch: 45
Number of 2s in fuzzymatch: 2
Number of 3s in fuzzymatch: 0
Number of non-matches in fuzzymatch: 250


In [8]:
def get_correct_compustathassan_clean_firm_name(row):
    best_choice = row['best_choice']
    if best_choice == 1:
        return row['choice_1_name']
    elif best_choice == 2:
        return row['choice_2_name']
    elif best_choice == 3:
        return row['choice_3_name']
    else:
        return math.nan

In [9]:
# Get the correct clean_firm_name from compustathassan based on the choice ('company_name')
fuzzymatch['clean_firm_name_compustathassan'] = fuzzymatch.apply(get_correct_compustathassan_clean_firm_name, axis=1)
fuzzymatch

,clean_firm_name_entryfile,count,choice_1_name,choice_1_score,choice_2_name,choice_2_score,choice_3_name,choice_3_score,best_choice,clean_firm_name_compustathassan
0,EUROPEAN RELIANCE GENERAL INSURANCE CO S A,1,EUROPEAN RELIANCE GENERAL INSURANCE CO SA,98.795181,A E PLASTIK PAK INC,85.500000,A M FOOD SERVICES INC,85.500000,1.0,EUROPEAN RELIANCE GENERAL INSURANCE CO SA
1,GENWORTH MORTGAGE INSURANC AUSTRALIA LTD,1,GENWORTH MORTGAGE INSURANCE AUSTRALIA LTD,98.765432,AUSTRALIA,90.000000,CB AUSTRALIA LTD,87.096774,1.0,GENWORTH MORTGAGE INSURANCE AUSTRALIA LTD
2,CALUMET SPECIALTY PRODUCTS PARTNERS L P,1,CALUMET SPECIALTY PRODUCTS PARTNERS LP,98.701299,ALLIED PRODUCTS CORP,85.500000,ALLOY METAL PRODUCTS INC,85.500000,1.0,CALUMET SPECIALTY PRODUCTS PARTNERS LP
3,BROOKFIELD INFRASTRUCTURE PARTNERS L P,10,BROOKFIELD INFRASTRUCTURE PARTNERS LP,98.666667,BROOKFIELD INFRASTRUCTURE CORP,86.363636,B P OIL GAS LTD,85.500000,1.0,BROOKFIELD INFRASTRUCTURE PARTNERS LP
4,SHIN KONG FINANCIAL HOLDINGS CO LTD,2,SHIN KONG FINANCIAL HOLDING CO LTD,98.550725,ZB FINANCIAL HOLDINGS LTD,88.936170,SINOPAC FINANCIAL HOLDINGS CO LTD,88.235294,1.0,SHIN KONG FINANCIAL HOLDING CO LTD
...,...,...,...,...,...,...,...,...,...,...
292,GRIFFIN INDUSTRIAL REALTY INC,1,A E PLASTIK PAK INC,85.500000,AA IMPORTING CO INC,85.500000,ABA INDUSTRIES INC,85.500000,NaN,NaN
293,INTL MEAL CO HOLDINGS SA,1,SOLIGOR INTL INC,85.500000,AMCA INTL CORP,85.500000,ANR PIPELINE CO,85.500000,NaN,NaN
294,ERAMETSLN,1,ERAMET SA,77.777778,ESR,72.000000,VELOCITYSHARES DAILY INVERSE VIX SHORTTERM ETN,67.500000,NaN,NaN
295,ZEBU,9,ZEBRA TECHNOLOGIES CORP,77.142857,EBUX INC,77.142857,ZEBALLOS MINING CO,77.142857,NaN,NaN


In [10]:
# Get the relevant columns from fuzzymatch
fuzzymatch = fuzzymatch[['clean_firm_name_entryfile', 'count', 'clean_firm_name_compustathassan']]
fuzzymatch

,clean_firm_name_entryfile,count,clean_firm_name_compustathassan
0,EUROPEAN RELIANCE GENERAL INSURANCE CO S A,1,EUROPEAN RELIANCE GENERAL INSURANCE CO SA
1,GENWORTH MORTGAGE INSURANC AUSTRALIA LTD,1,GENWORTH MORTGAGE INSURANCE AUSTRALIA LTD
2,CALUMET SPECIALTY PRODUCTS PARTNERS L P,1,CALUMET SPECIALTY PRODUCTS PARTNERS LP
3,BROOKFIELD INFRASTRUCTURE PARTNERS L P,10,BROOKFIELD INFRASTRUCTURE PARTNERS LP
4,SHIN KONG FINANCIAL HOLDINGS CO LTD,2,SHIN KONG FINANCIAL HOLDING CO LTD
...,...,...,...
292,GRIFFIN INDUSTRIAL REALTY INC,1,NaN
293,INTL MEAL CO HOLDINGS SA,1,NaN
294,ERAMETSLN,1,NaN
295,ZEBU,9,NaN


In [11]:
# , validate='1:1'
fuzzymatch = fuzzymatch.merge(compustathassan, how='inner', on='clean_firm_name_compustathassan')
fuzzymatch['exactmatch'] = 0
fuzzymatch

,clean_firm_name_entryfile,count,clean_firm_name_compustathassan,gvkey,company_name,hqcountry,exactmatch
0,EUROPEAN RELIANCE GENERAL INSURANCE CO S A,1,EUROPEAN RELIANCE GENERAL INSURANCE CO SA,243128.0,European Reliance General Insurance Co SA,GR,0
1,GENWORTH MORTGAGE INSURANC AUSTRALIA LTD,1,GENWORTH MORTGAGE INSURANCE AUSTRALIA LTD,317594.0,Genworth Mortgage Insurance Australia Ltd,AU,0
2,CALUMET SPECIALTY PRODUCTS PARTNERS L P,1,CALUMET SPECIALTY PRODUCTS PARTNERS LP,165846.0,"Calumet Specialty Products Partners, L.P.",US,0
3,BROOKFIELD INFRASTRUCTURE PARTNERS L P,10,BROOKFIELD INFRASTRUCTURE PARTNERS LP,179216.0,Brookfield Infrastructure Partners LP,BM,0
4,SHIN KONG FINANCIAL HOLDINGS CO LTD,2,SHIN KONG FINANCIAL HOLDING CO LTD,223107.0,Shin Kong Financial Holding Co Ltd,TW,0
5,SPORTSMAN S WAREHOUSE HOLDINGS INC,1,SPORTSMANS WAREHOUSE HOLDINGS INC,16009.0,Sportsmans Warehouse Holdings Inc,US,0
6,BROOKFIELD PROPERTY PARTNERS L P,1,BROOKFIELD PROPERTY PARTNERS LP,17556.0,Brookfield Property Partners L.P.,BM,0
7,AREZZO INDUSTRIA E COMERCIO S A,4,AREZZO INDUSTRIA E COMERCIO SA,296738.0,Arezzo Industria e Comercio SA,BR,0
8,COOPER STANDARD HOLDINGS INC,10,COOPERSTANDARD HOLDINGS INC,163765.0,Cooper-Standard Holdings Inc.,US,0
9,COLUMBIA BANKING SYSTEMS INC,1,COLUMBIA BANKING SYSTEM INC,25376.0,Columbia Banking System Inc,US,0


In [25]:
# merge to get the gvkey,company_name, hqcountry


In [ ]:
# save that.
df = pd.concat([exactmatch, fuzzymatch])


In [21]:

# Then finally append gvkey and country to the firm names